<a href="https://www.kaggle.com/code/arjav007/tb-image-classification-pytorch?scriptVersionId=233422539" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# **Deep Learning Based Chest X-ray Image Classification: TB Positive vs Negative**
---

### **Overview of Tuberculosis (TB)**

**According to the [World Health Organization (WHO)](https://www.who.int/health-topics/tuberculosis#tab=tab_1):**
- Tuberculosis is caused by the bacterium *Mycobacterium tuberculosis* and most commonly affects the lungs.
- TB spreads through the air when individuals with active lung TB cough, sneeze, or spit. Inhalation of only a few germs can lead to infection.
- Chest X-rays of patients with advanced TB often reveal **lung infections** and **cavity formation**.

![TB Chest X-ray](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTWfVD5iIMNyJdAYNMayyhK6q3WbfBxynoH_w&usqp=CAU)

> *Image Source: [radlines.org](https://radlines.org/X-ray_of_the_thorax_in_tuberculosis)*

---

### **Role of Deep Learning in Tuberculosis Detection**

- **Automated Diagnosis from Chest X-rays:**  
  Deep learning models—particularly Convolutional Neural Networks (CNNs)—can be trained to analyze chest X-ray images and classify them as TB-positive or TB-negative. This automation helps reduce the diagnostic workload for radiologists.

- **Early Detection and Consistency:**  
  Deep learning systems can detect subtle radiographic features in early-stage TB that might be missed by human experts. They also provide consistent results, unaffected by fatigue or human bias.



---
# **EDA and Data Loading**

In [ ]:
from collections import Counter
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader, random_split
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, roc_curve
from tqdm import tqdm
import pickle

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

## *Load Images and Labels*

The **`datasets`** module from **`torchvision`** provides a convenient utility called **`ImageFolder`** for loading image data that is organized in a specific directory structure:

**root/dog/xxx.png \
root/dog/xxy.png \
root/dog/[...]/xxz.png \
.
.
. \
root/cat/123.png \
root/cat/nsdf3.png \
root/cat/[...]/asd932_.png**



In this format, each subdirectory (e.g., `dog`, `cat`) represents a separate class, and all images within that folder are automatically associated with the corresponding class label.

We will use **`ImageFolder`** to load our dataset, as it automatically maps each image to its appropriate label based on the folder name.

For our task:
- **`0` → TB Negative**
- **`1` → TB Positive**

This makes dataset loading both efficient and intuitive for classification tasks.


In [ ]:
# Define the root directory
root = "/kaggle/input/TB_Chest_Radiography_Database"

# Define data transformations (modify as needed)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to a fixed size
    transforms.ToTensor(),          # Convert images to tensors
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize (modify based on dataset)
])

# Load the dataset
dataset = datasets.ImageFolder(root, transform=transform)

## *Split data for train val and test dataset*

In [ ]:
# Define split ratios
train_ratio, val_ratio, test_ratio = 0.8, 0.1, 0.1  # 80% train, 10% val, 10% test
total_size = len(dataset)
train_size = int(train_ratio * total_size)
val_size = int(val_ratio * total_size)
test_size = total_size - train_size - val_size  # Ensure all data is used

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Print dataset sizes
print(f"Train size: {len(train_dataset)}, Validation size: {len(val_dataset)}, Test size: {len(test_dataset)}")

## *Display sample images from training dataset*

In [ ]:
# Function to denormalize and convert tensor to numpy image
def denormalize(img_tensor, mean=0.5, std=0.5):
    img = img_tensor.numpy().transpose((1, 2, 0))  # (C, H, W) → (H, W, C)
    img = img * std + mean
    return np.clip(img, 0, 1)

# Find one Normal and one Tuberculosis image
normal_img = None
tb_img = None
for img_tensor, label in train_dataset:
    if label == 0 and normal_img is None:
        normal_img = (img_tensor, label)
    elif label == 1 and tb_img is None:
        tb_img = (img_tensor, label)
    if normal_img and tb_img:
        break

# Prepare images and labels
images = [normal_img[0], tb_img[0]]
labels = [normal_img[1], tb_img[1]]
class_names = dataset.classes  # ['Normal', 'Tuberculosis']

# Plot side by side
fig, axes = plt.subplots(1, 2, figsize=(8, 4))
for i in range(2):
    img = denormalize(images[i])
    axes[i].imshow(img.squeeze(), cmap='gray')
    axes[i].set_title(f"Label: {class_names[labels[i]]}")
    axes[i].axis("off")

plt.tight_layout()
plt.show()


## *Display Training Image distribution: TB positive and negative*

In [ ]:
# Count the number of samples per class
class_counts = Counter([label for _, label in dataset.samples])

# Get class names and counts
class_names = dataset.classes  # Class names from ImageFolder
counts = [class_counts[i] for i in range(len(class_names))]

# Plot the class distribution
plt.figure(figsize=(8, 6))
plt.bar(class_names, counts, color='skyblue')
plt.xlabel("Class")
plt.ylabel("Number of Images")
plt.title("Class Distribution in Dataset")
plt.xticks(rotation=45)  # Rotate labels if needed
plt.grid(axis='y', linestyle="--", alpha=0.7)
plt.show()

tb_neg_images = counts[0]
tb_pos_images = counts[1]
print('Percent of Tuberculosis Positive : {:.2f} %'.format(100 * tb_pos_images/(tb_neg_images+tb_pos_images)))
print('Percent of Tuberculosis Negative : {:.2f} %'.format(100 * tb_neg_images/(tb_neg_images+tb_pos_images)))

## *Handling Class Imbalance*

**Sample Weighting in Loss Function (weighting the contributions of positive and negative losses to be equal)**

create positive/negative contributors
$$w_{pos} \times freq_{p} = w_{neg} \times freq_{n},$$
$$w_{pos} = freq_{neg}$$
$$w_{neg} = freq_{pos}$$

Final weight loss
$$\mathcal{L}_{cross-entropy}^{w}(x) = - (w_{p} y \log(f(x)) + w_{n}(1-y) \log( 1 - f(x) ) ).$$

In [ ]:
pos_freq = round(tb_pos_images/tb_neg_images, 2)
neg_freq = 1 - pos_freq

pos_weights = neg_freq
neg_weights = pos_freq

pos_contribution = pos_freq * pos_weights 
neg_contribution = neg_freq * neg_weights

In [ ]:
# Convert weights to torch tensor
class_weights = torch.tensor([neg_weights, pos_weights], dtype=torch.float32)

# Define loss function
criterion = nn.CrossEntropyLoss(weight=class_weights)

# Training the Model

*We will be using gradient accumulation to make the training more robust and faster. It helps in simulating the larger batch sizes while handling the memory contraints. More information can be found [here](https://kozodoi.me/blog/20210219/gradient-accumulation)*

In [ ]:
# Hyperparameters
EPOCHS = 5
LR = 1e-5
BATCH_SIZE = 128
MAX_PHYSICAL_BATCH_SIZE = 16
ACCUM_ITER = BATCH_SIZE // MAX_PHYSICAL_BATCH_SIZE

## *Creating the data loaders*

Since gradient accumulation will be used during training, the batch size specified in the training data loader should be less than or equal to the maximum batch size that the GPU can handle in a single forward and backward pass. This ensures that memory constraints are respected while still allowing us to simulate larger effective batch sizes through accumulation over multiple steps.

In [ ]:
train_loader = DataLoader(train_dataset, batch_size = MAX_PHYSICAL_BATCH_SIZE, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = MAX_PHYSICAL_BATCH_SIZE, shuffle = False)
test_loader = DataLoader(test_dataset, batch_size = MAX_PHYSICAL_BATCH_SIZE, shuffle = False)

## *Define Training Function*

In [ ]:
def train(model, train_loader, optimizer, epoch, device, accum_iter):
    model.train()
    criterion = nn.CrossEntropyLoss()

    losses = []
    top1_acc = []
    
    optimizer.zero_grad()

    for i, (images, target) in enumerate(train_loader):   
        images = images.to(device)
        target = target.to(device)

        with torch.set_grad_enabled(True):
            # compute output
            output = model(images)
            loss = criterion(output, target)

            preds = np.argmax(output.detach().cpu().numpy(), axis=1)
            labels = target.detach().cpu().numpy()

            # measure accuracy and record loss
            acc = accuracy_score(labels, preds)
            losses.append(loss.item())
            top1_acc.append(acc)

            # gradient accumulation
            loss = loss / accum_iter
            loss.backward()

            if (i + 1) % accum_iter == 0 or (i + 1 == len(train_loader)):
                optimizer.step()
                optimizer.zero_grad()

        if (i + 1) % 200 == 0:
            print(
                f"\tTrain Epoch: {epoch} \t"
                f"Loss: {np.mean(losses):.6f} "
                f"Acc@1: {np.mean(top1_acc) * 100:.6f}"
            )


## *Define Evaluation Function*

In [ ]:
def evaluate(model, test_loader, device, num_classes=2):
    model.eval()
    criterion = nn.CrossEntropyLoss()
    losses = []
    top1_acc = []
    all_preds = []
    all_labels = []
    all_probs = []

    with torch.no_grad():
        for images, target in test_loader:
            images = images.to(device)
            target = target.to(device)

            output = model(images)
            loss = criterion(output, target)
            probs = torch.softmax(output, dim=1).cpu().numpy()
            preds = np.argmax(probs, axis=1)
            labels = target.cpu().numpy()

            acc = accuracy_score(labels, preds)
            losses.append(loss.item())
            top1_acc.append(acc)
            all_preds.extend(preds)
            all_labels.extend(labels)
            all_probs.extend(probs)

    top1_avg = np.mean(top1_acc)
    mean_loss = np.mean(losses)
    f1 = f1_score(all_labels, all_preds, average="macro")
    auc = roc_auc_score(all_labels, all_probs, multi_class="ovr") if num_classes > 2 else roc_auc_score(all_labels, [p[1] for p in all_probs])

    return top1_avg, f1, auc, all_labels, all_probs, mean_loss

## *Training Loop*

In [ ]:
name = 'DenseNet121'
print(f"The Model is: {name}")

# Call the model
model = models.densenet121(num_classes=2)
model = model.to(device)

# Call the optimizer
optimizer = optim.RMSprop(model.parameters(), lr=LR) 

# Training the model
for epoch in range(EPOCHS):
    train(model, train_loader, optimizer, epoch + 1, device, ACCUM_ITER)
    print(f"Evaluating the model")
    top1_avg, f1, auc_score, all_labels, all_probs, mean_loss = evaluate(model, val_loader, device)
    print(
        f"\tVal set:"
        f" Loss: {mean_loss:.6f} "
        f" Acc: {top1_avg * 100:.6f} "
        f" F1: {f1:.6f} "
        f" AUC: {auc_score:.6f} "
    )
    print(f"-------------------------------------")

# **Evaluating the Model on Test Data**

To assess the model’s performance on unseen data, we use three important evaluation metrics:

1. **Accuracy –** Measures the overall correctness of the model by calculating the proportion of total predictions that were correct.
2. **F1 Score –** Provides a balance between precision and recall, especially useful when dealing with imbalanced datasets, as it considers both false positives and false negatives.
3. **AUC-ROC Score –** Evaluates the model’s ability to distinguish between classes by measuring the trade-off between the true positive rate and the false positive rate across different threshold values.

Together, these metrics offer a comprehensive view of the model’s effectiveness in handling the classification task.

In [ ]:
# Evaluating the model
top1_avg, f1, auc_score, all_labels, all_probs, mean_loss = evaluate(model, test_loader, device)
print(
        f" Val set:"
        f" Loss: {mean_loss:.6f} "
        f" Acc: {top1_avg * 100:.6f} "
        f" F1: {f1:.6f} "
        f" AUC: {auc_score:.6f} "
    )
print(f"-------------------------------------")

# Plot ROC Curve for this batch size
plt.figure(figsize=(10, 8))
fpr, tpr, _ = roc_curve(all_labels, [p[1] for p in all_probs])  
plt.plot(fpr, tpr, label=f"AUC = {auc_score:.4f}")
plt.plot([0, 1], [0, 1], linestyle="--", color="gray")
plt.title(f"AUC ROC Curve")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.show()

---

# **Conclusion**

This notebook demonstrated a deep learning-based approach to classify chest X-ray images for Tuberculosis (TB) detection. By leveraging Convolutional Neural Networks and appropriate evaluation metrics, we were able to build an effective model that can aid in the early and automated diagnosis of TB.

While the model demonstrates strong performance and is well-suited for the task at hand, there's always room for further improvement. There are several strategies that could be explored to further enhance its effectiveness:

1. **Reduce the Batch Size** – A smaller batch size might help the model generalize better by introducing more gradient noise during training, which can lead to improved performance on unseen data.
2. **Increase the Number of Epochs with a Learning Rate Scheduler** – Training the model for more epochs while using a learning rate scheduler can allow it to learn more thoroughly without overfitting, by gradually reducing the learning rate as training progresses.
3. **Experiment with Larger Models** – Trying architectures with more parameters and deeper layers could potentially improve performance, especially if the task benefits from higher model capacity.
4. **Perform Hyperparameter Tuning** – Systematically optimizing hyperparameters (like learning rate, dropout rate, and optimizer settings) could lead to better model performance by finding the most effective configuration

Automating TB detection using Deep Learning not only supports radiologists but also has the potential to make healthcare more accessible and scalable—especially in resource-limited settings.

**Thank you for exploring this notebook!**
